In [1]:
import os
import pickle
import uuid
import glob 
from tqdm import tqdm
import numpy as np
from PIL import Image
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.utils.modanetDrawer_test import ModaNetDrawerTest
from maskrcnn_benchmark.utils.viz_utils import *
from PIL import Image
from pycocotools.mask import encode, decode
import matplotlib.pyplot as plt

In [2]:
categories = [
    "__background",
    "bag",
    "belt",
    "outer",
    "dress",
    "sunglasses",
    "pants",
    "top",
    "shorts",
    "skirt",
    "headwear",
    "scarf & tie"
]

In [3]:
cfg.merge_from_file('../configs/imaterialist+modanet-8gpu-COCOft.yaml')
cfg.merge_from_list(['MODEL.WEIGHT', '/data/logs/garment_detection/imaterialist+modanet_continue/model_0047500.pth'])
cfg.merge_from_list(['TEST.IMS_PER_BATCH', '2'])
drawer = ModaNetDrawerTest(cfg, min_image_size=800, confidence_threshold=0.5)

In [4]:
images = glob.glob('/data/runway2shop/looks/*.jpg')

In [13]:
SAVEDIR = '/data/runway2shop/maskrcnn_preds/looks/'

In [16]:
for image_path in tqdm(images) :
    image = np.array(Image.open(image_path).convert('RGB'))
    top_predictions, features = drawer.run_on_opencv_image(image)

    bboxes = top_predictions.bbox.numpy()
    masks = top_predictions.get_field("mask").numpy()
    labels = top_predictions.get_field("labels").numpy().astype(int)
    masks = masks.squeeze(1).transpose((1,2,0))
    scores = top_predictions.get_field("scores").numpy()
    
    features = features.numpy()
    
    image_name = os.path.basename(image_path).split('.')[0]
    
    savefolder = os.path.join(SAVEDIR, image_name)
    if not os.path.exists(savefolder) :
        os.makedirs(savefolder)
        
    np.save(os.path.join(savefolder, 'bbox.npy'), bboxes)
    np.save(os.path.join(savefolder, 'masks.npy'), masks)
    np.save(os.path.join(savefolder, 'labels.npy'), labels)
    np.save(os.path.join(savefolder, 'scores.npy'), scores)
    np.save(os.path.join(savefolder, 'features.npy'), features)

100%|██████████| 116/116 [04:09<00:00,  2.32s/it]


In [7]:
masks.shape

(4098, 2732, 3)

In [11]:
os.path.basename(image_path).split('.')[0]

'0071'